# Algoritmos Genéticos

**Professor:**
Cristiano Leite de Castro

**Alunos:**
Gabriel Camatta Zanotelli	2018020140
Lucas de Almeida Martins	2018020328

## Introdução

O problema proposto neste trabalho consiste no desenvolvimento de um algoritmo genético que que evolui uma população de strings até convergir para a string “METHINKS*IT*IS*LIKE*A*WEASEL”.

No caso, a probabilidade de uma string gerada aleatoriamente ser igual à string desejada é de 1/(38^23), o que é uma probabilidade extremamente baixa. Utilizar um algoritmo evolucionário com os parâmetros corretos contorna esse problema, pois ele não depende que a um indivíduo muito apto esteja presente na população inicial, pois ela é composta por vários indivíduos que vão se reproduzir e gerar mutações.

## Implementação

### Definições de variáveis iniciais

In [ ]:
import random as rd
import matplotlib.pyplot as plt
import string as str

In [ ]:
crossover_rate = 0.5
mutation_rate = 0.2

base_population = 20
pop_sample = 5

FINAL_TEXT = list("METHINKS*IT*IS*LIKE*A*WEASEL")
text_size = len(FINAL_TEXT)

VALID_CHARS = list(str.ascii_uppercase + str.digits + '*')

fitness_history = []


### Funções de suporte

In [ ]:
def argsort(seq):
    return sorted(range(len(seq)), key=seq.__getitem__)

def verify_condition(population):
    return len(FINAL_TEXT) in population

def get_best_solution(population_fit):
    count = 0
    aux = 0
    for i in range(len(population_fit)):
        if population_fit[i] > aux:
            aux = population_fit[i]
            count = i
    return count

def join(vector):
    word = ""
    for c in vector:
        word += c
    return word

def check_crossover():
    return rd.random() < crossover_rate

def check_mutation():
    return rd.random() < mutation_rate

def print_initial_pop(population, _print=True):
    if _print:
        print("População inicial:")
        for i in range(int(base_population / 4)):
            print(join(population[i]), " - ", join(population[i+1]), " - ",
                  " - ", join(population[i+2]), " - ", join(population[i+3]))
        print()

def print_initial_pop_fitt(population_fitt, _print=True):
    if _print:
        print("Fittness da população inicial:")
        print(population_fitt)
        print()

def print_final_results(pop, population_fitt, count_gen, _print=True):
    best_pop = pop[get_best_solution(population_fitt)]
    if _print:
        print("--------------------RESULTADO--------------------")
        print("Geração final: ", (count_gen+1))
        print()
        print("Fittness da população final:")
        print(population_fitt)
        print()
        print("Melhor indivíduo: ", join(best_pop))
        print("Texto objetivo  : ", join(FINAL_TEXT), "- Acertos: ", population_fitt[pop.index(best_pop)])
    else:
        print("Número de gerações:", count_gen+1, " | Acertos:", population_fitt[pop.index(best_pop)])

### Definição da População Inicial

Para a determinação dos indivíduos da população inicial, utiliza-se uma função que preenche, com caracteres aleatórios, uma lista de caracteres do mesmo tamanho que a string final. Esses caracteres são separados pelo marcador '*' e podem ser letras maiúsculas e dígitos.


In [ ]:
def init_population(_base_pop = 20):
    population = []
    for i in range (_base_pop):
        word = [rd.choice(VALID_CHARS) for _ in range(text_size)]
        population.append(word)
    return population

### Avaliação de indivíduos

A avaliação do *fitness* de um indivíduo é dada por uma função que compara caractere por caractere do invidíduo com a string final, cada caractere igual conta como um acerto, de forma que a quantidade de acertos final determina o *fitness* do indivíduo.

In [ ]:
def fitness_nq(solution, _b=False):
    checks = 0
    for i in range(len(solution)):
        if solution[i] == FINAL_TEXT[i]:
            checks+=1
    return checks

### Seleciona pais

Foi selecionado o método de Roleta para a seleção dos pais, que consiste na maior probabilidade de escolha dos indivíduos com  maior *fitness* da população. No caso da implementação abaixo, a soma dos *fitness* de todos indivíduos será a escala da roleta, a partir da qual serão escolhidos dois pontos aleatoriamente e de acordo com esses pontos, os indivíduos serão selecionados


In [ ]:
def select_parents_roulete(population, population_fitt):
    parents = [None, None]

    total = sum(population_fitt)
    p1 = rd.randint(1, total)
    p2 = rd.randint(1, total)

    count = 0
    prev = 0
    for i in range(len(population)):
        count += population_fitt[i]
        if (prev < p1 <= count) and parents[0] is None:
            parents[0] = population[i]
        if (prev < p2 <= count) and parents[1] is None:
            parents[1] = population[i]
        prev = count

    return parents

### Recombinação dos pais

Utilização de um método de *Cut and crossfill*, no qual os pais são “cortados” em um ponto aleatório e recombinados, de forma que o filho 1 é formado pela primeira parte do pai 1 e a segunda parte do pai 2, e o filho 2 é formado pela a primeira parte do pai 2 e a primeira parte do pai 1.

In [ ]:
def cut_and_crossfill(parents):

    cross_point = rd.randint(1, text_size-1)

    c1 = parents[0][:cross_point] + parents[0][cross_point:]
    c2 = parents[1][:cross_point] + parents[1][cross_point:]

    return [c1, c2]

### Mutação dos filhos

A mutação dos filhos ocorrerá por um parâmetro que varia entre 0 e 100%. Além disso, como no caso desse problema é desejável que um caractere correto permaneça em sua posição e sem alterações, dessa forma, condicionamos a mutação para que ela não ocorra em caracteres que já estão iguais com o da string final.


In [ ]:
def mutate_offspring(offspring):
    for of in offspring:
        for c in range(len(of)):
            if check_mutation() and of[c] != FINAL_TEXT[c]:
                of[c] = rd.choice(VALID_CHARS)
    return offspring

## Cria e seleciona nova geração

Uma nova geração consiste na geração atual juntamente com os filhos gerados, totalizando numa população temporária com o tamanho de   “tamanho da população” + 2. Após isso, os dois indivíduos com menor *fitness* são descartados da nova população.


In [ ]:
def select_new_generation(poulation, offspring):
    new_generation = poulation + offspring

    new_pop_fit = [0] * len(new_generation)
    for i in range(len(new_generation)):
        new_pop_fit[i] = fitness_nq(new_generation[i])

    new_pop_id = argsort(new_pop_fit)[len(offspring):]

    next_generation = []
    for i in range(len(new_generation)):
        if i in new_pop_id:
            next_generation.append(new_generation[i])

    return next_generation

### Desenha gráfico

Os gráficos gerados  do programa o possuem o eixo y como o *fitness* e o eixo x como geração e são traçadas duas curvas para a análise dos resultados, uma que representa a média da população e outra que representa o melhor indivíduo de cada população.

In [ ]:
def draw_graph(datax, datay, _print=True):
    if _print:
        plt.plot(range(datax+1), datay[0], "-g", label="Medium")
        plt.plot(range(datax+1), datay[1], "-r", label="Best")
        plt.legend(loc="upper right")
        plt.xlabel('Generation')
        plt.ylabel('Fittness')
        plt.show()

## Criação do Métodos

In [ ]:
def run_gen_alg(population, _max_generations=1000, _base_pop=20,
                _crossover_rate=0.5, _mutation_rate=0.2, _print=True):

    # Redefine variáveis usando argumentos
    max_generation = _max_generations
    base_population = _base_pop
    crossover_rate = _crossover_rate
    mutation_rate = _mutation_rate
    print = _print

    # Fittnes inicial
    population_fitt = [0] * base_population
    for i in range(base_population):
        population_fitt[i] = fitness_nq(population[i])

    # Prints iniciais
    print_initial_pop(population, _print=print)
    print_initial_pop_fitt(population_fitt, _print=print)

    # Roda o algorítmo
    count_gen = 0
    datay = [[], []]
    for i in range(max_generation):

        parents = select_parents_roulete(population, population_fitt)

        offspring = cut_and_crossfill(parents)

        mutate_offspring(offspring)

        population = select_new_generation(population, offspring)

        for j in range(base_population):
            population_fitt[j] = fitness_nq(population[j])

        datay[0].append(sum(population_fitt) / len(population_fitt))
        datay[1].append(max(population_fitt))

        count_gen = i
        if verify_condition(population_fitt):
            break

    best = population[get_best_solution(population_fitt)]

    # Análise dos dados finais
    print_final_results(population, population_fitt, count_gen, _print=print)

    # Impressão do gráfico
    draw_graph(count_gen, datay, _print=print)

    return best


In [ ]:

population = init_population()

best = run_gen_alg(population)


## Resultados

O programa foi feito de forma a considerar um conjunto de parâmetros padrões, escolhidos arbitrariamente, e que serão assumidos quando não comentados nos testes a seguir. Dentre eles, temos:
- População inicial: 20
- Máximo de gerações: 1000
- Probabilidade de *crossover* (cruxamento): 50%
- Probabilidade de mutação: 20%

### Verificação de convergência

Utilizamos as configurações padrões da função desenvolvida para rodar o problema 30 vezes, de forma a obter uma média de convergência.

Com essas configurações, foi obtido uma média de 634 gerações para que se econtrasse o resultado final, com todos os casos chegando ao resultado final antes do limite padrão estipulado.

No entanto, vale ressaçtar que há uma alta variação na função, populações que encontram um resultado final após 498 gerações, enquantos outras levam 827, no caso das populações geradas para essa análise.

In [ ]:
for i in range(30):
    population = init_population()
    best = run_gen_alg(population,
                       _print=False)

### Variação da Probabilidade de Mutação

Para a análise do impacto da taxa de mutação na função desenvolvida, a função abaixo foi criada de forma a iterar por diversas taxas diferentes, começando com uma probabilidade de 5% e caminhando de 5 em 5% até um máximo de 95% de probabilidade de mutação em cada posição do vetor.

Foi constatado que o método desenvolvido, mesmo instável, se mantém mais eficiente quando se aplica uma taxa de mutação entre 15% e 30%, considerando os demais parametros utilizados. Temos então uma perda de eficiência progressiva conforme aumentamos esse parâmetro.

In [ ]:
mutation_rate_var = 0.05
while mutation_rate_var <= 1:
    print("Crossover rate: ", round(mutation_rate_var, 2))
    population = init_population()
    best = run_gen_alg(population,
                       _print=False,
                       _mutation_rate=mutation_rate_var)
    mutation_rate_var += 0.05

## Conclusão

Antes de condicionar a mutação para ocorrer apenas nas posições que o indivíduo se distinguia da string final, a quantidade de gerações necessárias para atingir para atingir a string final era 10 vezes maior que o valor médio de gerações apresentado nos resultados. Isso provou que, no campo de computação evolucionária, adotar diferentes abordagens para problemas parecidos pode faazer com que se gaste muito menos poder computacional para obter uma resposta adequada.
